# Tutorials i TensorFlow fra Kieth Downing
## Sett 2, Autoencoder.

Dette eksempelet er et neuralt nett som baserer seg på bits. Den skal få inn bits, så gjennom et hemmlig lag bestående av færre noder enn inputverdier skal den finne tilbake til bits som var i input.

Autoencoder er her laget som en egen klasse. Denne klassen kan bygge det neurale nettet med gitte parametere som spesifiserer hvordan nettet skal se ut. 

Keith downings forklarende tekst:

We can extend the basic approach in tfex8 (tutor1.py) to handle 
1. A 3-layered neural network
2. A collection of cases to be learned.  

This is a specialized neural network designed to solve one type of classification problem: converting an input string, through a single hidden layer, to a copy of itself on the output end.


In [1]:
## HENTET FRA autoencoder():
def __init__(self,nh=3,lr=.1):
    self.cases = TFT.gen_all_one_hot_cases(2**nh)
    self.learning_rate = lr
    self.num_hiddens = nh
    self.build_neural_network(nh)

Her brukes det to variabler som input for metoden. Disse to skal spesifisere nettet. 

```python
	nh = 3   # number of hidden nodes in the hidden layer
	lr = 0.2 # learning rate used with gradient decent.
```

Casene hentes fra en genereringsmetode laget av downing. Denne lager bare bit-strenger. Altså sekvenser med bits. Her kommer sekvensene i lengde 2*nh. De hver sekvens er altså dobbelt så lage som antall hidden nodes.

Hvis det er tre noder i det hemmelige laget, kommer input i form av:
```
	011001
```

## BYGGING AV DET NEURALE NETTET:
Under kommer forklaring av koden i metoden "build_neural_network()".

In [3]:
def build_neural_network(self,nh):
    ios = 2**nh  # ios = input- and output-layer size
    self.w1 = tf.Variable(np.random.uniform(-.1,.1,size=(ios,nh)),name='Weights-1')  # first weight array
    self.w2 = tf.Variable(np.random.uniform(-.1,.1,size=(nh,ios)),name='Weights-2') # second weight array
    self.b1 = tf.Variable(np.random.uniform(-.1,.1,size=nh),name='Bias-1')  # First bias vector
    self.b2 = tf.Variable(np.random.uniform(-.1,.1,size=ios),name='Bias-2')  # Second bias vector
    self.input = tf.placeholder(tf.float64,shape=(1,ios),name='Input')
    self.target = tf.placeholder(tf.float64,shape=(1,ios),name='Target')
    self.hidden = tf.sigmoid(tf.matmul(self.input,self.w1) + self.b1,name="Hiddens")
    self.output = tf.sigmoid(tf.matmul(self.hidden,self.w2) + self.b2, name = "Outputs")
    self.error = tf.reduce_mean(tf.square(self.target - self.output),name='MSE')
    self.predictor = self.output  # Simple prediction runs will request the value of outputs
    # Defining the training operator
    optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
    self.trainer = optimizer.minimize(self.error,name='Backprop')

Det bortgjemte laget skal alltid være halve størrelsen av input - og output lagene. 
```
ios = 2**nh
```

Han definerer 2 forskjellige sett med vekter her. Det må to sett til fordi det er et hidden-layer mellom input og output. All kommunikasjon skjer gjennom vektmatrisene.